In [21]:
import os
import numpy as np
import cv2
from PIL import Image
import json

In [22]:
path_main = "./../../../dataset_processed"
path_img = os.path.join(path_main, "fundus_image")
path_anot = os.path.join(path_main, "annotations")
path_mask = "./../../../dataset_used/mask_image"
classes = ["glaucoma", "non_glaucoma"]

In [23]:
annots = {}
for class_type in classes:
    annots[class_type] = json.load(open([os.path.join(path_anot,
                                                        class_type,
                                                        file)
                                            for file in os.listdir(os.path.join(path_anot,
                                                                                class_type))
                                            if file.endswith(".json")][0]))

In [31]:
for class_type, annotations in annots.items():
    for annotation in annotations:
        if len(annotation["annotation"]) < 2:
            continue
        b_mask = np.zeros((int(annotation["metadata"]["img_height"]),
                            int(annotation["metadata"]["img_width"])), dtype=np.uint8)
        
        for polygon in sorted(annotation["annotation"], key=lambda x: x["label"], reverse=True):
            label = 255 if polygon["label"] == "disc" else 64
            points = [list(map(np.float32, point.split(","))) for point in polygon["points"].split(";")]
            cv2.fillPoly(b_mask, [np.array(points, dtype=np.int32)], color=label)
        try:
            os.remove(os.path.join(path_mask, f"{annotation['metadata']['img_name'].split('.')[0]}.png"))
        except FileNotFoundError:
            pass
        Image.fromarray(b_mask).save(os.path.join(path_mask, f"{annotation['metadata']['img_name'].split('.')[0]}_mask.png"))